In [1]:
# add parent dir to syspath
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
root_dir = os.path.dirname(parent_dir)
sys.path.insert(0, root_dir)

In [2]:
from imageGenerators import load_realdata, distortions
from imageGenerators.imgGen_simple import synth_generator
from imageGenerators.imgGen_augmented import synth_generator as synth_generator_new

from models.cyclegan_modified.cyclegan import cyclegan


import tensorflow as tf
import cv2
from pathlib import Path
from matplotlib import pyplot as plt
import numpy as np
from time import time
import re
import random
import pickle

import lpips_interface

In [3]:
def showImage(image):
    if len(image.shape) == 3 and image.shape[2] == 1:
        image = image[:,:,0]
    image = image.astype(int)
    plt.imshow(image, cmap="gray", vmin=0, vmax=255)
    plt.show()
def showImages(images, n, labels=None):
    for i in range(min(n, len(images))):
        if not labels is None:
            print(labels[i])
        image = images[i]
        showImage(image)

In [4]:
fcsrn_height = 48; fcsrn_width = 160; fcsrn_channels = 1
fcsrn_shape = (fcsrn_height, fcsrn_width, fcsrn_channels)
fcsrn_dims = (fcsrn_width, fcsrn_height)

In [5]:
def prep(image):
    # norm to -1,1
    image = image.astype("float32")
    image = (image / 127.5) - 1
    # add channel-dims
    shape = image.shape
    shape = [d for d in shape]
    shape.append(1)
    if len(shape) == 3:
        # add batch-dimensions if single image
        shape.insert(0, 1)
    image = np.reshape(image, shape)
    # to 3-channel
    image = tf.tile(image, [1, 1, 1, 3])
    
    return image

In [6]:
n_pairs = 1900

In [7]:
def diversitiy(images, n_pairs):
    random_indices = np.random.randint(0,len(images), (2,n_pairs))
    images_a = images[random_indices[0]]
    images_b = images[random_indices[1]]
    
    images_a = prep(images_a)
    images_b = prep(images_b)
    
    stime = time()
    d = lpips_interface.distance(images_a,images_b)
    print("%.2f seconds" % (time() - stime) )
    
    mean = tf.math.reduce_mean(d)
    maximum = tf.math.reduce_max(d)
    minimum = tf.math.reduce_min(d)
    
    print("mean:\t%f" % (mean) )
    print("maximum:\t%f" % (maximum) )
    print("minimum:\t%f" % (minimum) )

In [8]:
# diversity real easy images with black padding
images, _ = load_realdata.load_wmr_easy(n_toLoad = None, resizeTo=fcsrn_dims, keepRatio=True)
diversitiy(images, n_pairs)

C:\Users\andre\jupyter_ws\imageGenerators\load_realdata.py:57: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(txt_path, sep=seperators ,header=None)


8.16 seconds
mean:	0.197358
maximum:	0.357190
minimum:	0.000000


In [9]:
# diversity real easy images without padding
images, _ = load_realdata.load_wmr_easy(n_toLoad = None, resizeTo=fcsrn_dims, keepRatio=False)
diversitiy(images, n_pairs)

0.53 seconds
mean:	0.188493
maximum:	0.323994
minimum:	0.000000


In [10]:
# diversity real diff images train with black padding
images, _ = load_realdata.load_wmr_diff_train(n_toLoad = None, resizeTo=fcsrn_dims, keepRatio=True)
diversitiy(images, n_pairs)

0.55 seconds
mean:	0.295067
maximum:	0.565676
minimum:	0.060789


In [11]:
# diversity real diff images train without black padding
images, _ = load_realdata.load_wmr_diff_train(n_toLoad = None, resizeTo=fcsrn_dims, keepRatio=False)
diversitiy(images, n_pairs)

0.53 seconds
mean:	0.282852
maximum:	0.522814
minimum:	0.046331


In [12]:
# diversity real diff images test with black padding
images, _ = load_realdata.load_wmr_diff_test(n_toLoad = None, resizeTo=fcsrn_dims, keepRatio=True)
diversitiy(images, n_pairs)

0.53 seconds
mean:	0.289920
maximum:	0.558719
minimum:	0.000000


In [13]:
# diversity real diff images test without black padding
images, _ = load_realdata.load_wmr_diff_test(n_toLoad = None, resizeTo=fcsrn_dims, keepRatio=False)
diversitiy(images, n_pairs)

0.53 seconds
mean:	0.283666
maximum:	0.491759
minimum:	0.000000
